In [1]:
# %%
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from keras import layers, models, regularizers, callbacks
from keras.api.optimizers import Adam
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt


# %%
# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load the dataset
df = pd.read_csv("./lucas_pre.csv")

# Display basic information
print(f"Dataset shape: {df.shape}")

# Identify column types
target = "pH_H2O"
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if target in numeric_cols:
    numeric_cols.remove(target)

categorical_cols = ["Depth", "LC", "LU", "USDA", "ISSS", "NUTS_0", "LC0_Desc", "LC1_Desc", "LU1_Desc"]
# Filter to only keep categorical columns that exist in the dataset
categorical_cols = [col for col in categorical_cols if col in df.columns]

# Remove columns with too many unique values or too many missing values
filtered_cat_cols = []
for col in categorical_cols:
    if col in df.columns:
        if df[col].nunique() < 30 and df[col].isna().sum() / len(df) < 0.3:
            filtered_cat_cols.append(col)

print(f"Using {len(numeric_cols)} numeric columns and {len(filtered_cat_cols)} categorical columns")

# Drop rows where target is missing
df = df.dropna(subset=[target])

# Feature engineering - create new features
if "EC" in df.columns and "OC" in df.columns:
    df["EC_OC_ratio"] = df["EC"] / df["OC"].replace(0, 0.001)

if "Clay" in df.columns and "Sand" in df.columns:
    df["Clay_Sand_ratio"] = df["Clay"] / df["Sand"].replace(0, 0.001)

if "N" in df.columns and "P" in df.columns and "K" in df.columns:
    # NPK balance is important for soil chemistry
    if df["N"].notnull().sum() > 0 and df["P"].notnull().sum() > 0 and df["K"].notnull().sum() > 0:
        df["NPK_sum"] = df["N"] + df["P"] + df["K"]

if "CaCO3" in df.columns:
    # Soil pH is strongly related to CaCO3
    df["log_CaCO3"] = np.log1p(df["CaCO3"])

if "OC" in df.columns and "N" in df.columns:
    # C:N ratio is important for soil biology
    df["CN_ratio"] = df["OC"] / df["N"].replace(0, 0.001)

if "Clay" in df.columns and "OC" in df.columns:
    # Clay-organic matter interactions affect pH
    df["Clay_OC_interaction"] = df["Clay"] * df["OC"]

# Extract target values
y = df[target].values

# Update numeric columns after adding engineered features
numeric_cols = [col for col in df.select_dtypes(include=[np.number]).columns.tolist() if col != target]

# Create preprocessing pipeline
numeric_transformer = Pipeline(
    steps=[
        ("imputer", KNNImputer(n_neighbors=5)),
        ("power", PowerTransformer(method="yeo-johnson")),  # Better than StandardScaler for skewed data
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(steps=[("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))])

# Combine all transformers
preprocessor = ColumnTransformer(
    transformers=[("num", numeric_transformer, numeric_cols), ("cat", categorical_transformer, filtered_cat_cols)]
)

# Prepare features
X = df[numeric_cols + filtered_cat_cols]

# Split data before preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Apply preprocessing
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print(f"Processed feature shape: {X_train_processed.shape}")


# %%
# Define a function to create the neural network model
def create_model(
    input_dim,
    neurons_layers=[128, 64, 32],
    dropout_rates=[0.4, 0.3, 0.2],
    activation="relu",
    learning_rate=0.001,
    l2_reg=0.001,
):
    model = models.Sequential()

    # Input layer
    model.add(layers.Input(shape=(input_dim,)))

    # Hidden layers
    for i, neurons in enumerate(neurons_layers):
        model.add(
            layers.Dense(
                neurons,
                activation=activation,
                kernel_regularizer=regularizers.l2(l2_reg),
                kernel_initializer="he_normal",
            )
        )
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dropout_rates[i]))

    # Output layer
    model.add(layers.Dense(1))

    # Compile
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["mae"])

    return model


# %%
# Create the model
model = create_model(
    input_dim=X_train_processed.shape[1],
    neurons_layers=[256, 128, 64, 32],  # Deeper network
    dropout_rates=[0.5, 0.4, 0.3, 0.2],
    activation="elu",  # ELU can work better than ReLU for some regression tasks
    learning_rate=0.0005,  # Lower learning rate for stability
    l2_reg=0.0005,
)

# Print model summary
model.summary()

# Define callbacks
early_stopping = callbacks.EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True, verbose=1)

reduce_lr = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=0.00001, verbose=1)

# %%
# Train the model with more epochs
history = model.fit(
    X_train_processed,
    y_train,
    validation_split=0.2,
    epochs=200,  # More epochs with early stopping
    batch_size=32,  # Try different batch sizes
    callbacks=[early_stopping, reduce_lr],
    verbose=1,
)

# %%
# Evaluate the model
loss, mae = model.evaluate(X_test_processed, y_test)
print(f"Test Mean Absolute Error: {mae:.4f}")

# Predict on test data
y_pred = model.predict(X_test_processed)
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2:.4f}")

# %%
# Plot training history
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Train", "Validation"], loc="upper right")

plt.subplot(1, 2, 2)
plt.plot(history.history["mae"])
plt.plot(history.history["val_mae"])
plt.title("Model MAE")
plt.ylabel("MAE")
plt.xlabel("Epoch")
plt.legend(["Train", "Validation"], loc="upper right")
plt.tight_layout()
plt.show()

# Plot predictions vs actual
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], "r--")
plt.xlabel("Actual pH")
plt.ylabel("Predicted pH")
plt.title(f"Actual vs Predicted pH (R² = {r2:.4f})")
plt.grid(True)
plt.tight_layout()
plt.show()


Dataset shape: (18984, 28)
Using 15 numeric columns and 7 categorical columns
Processed feature shape: (15186, 134)


2025-03-22 04:29:08.080634: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-03-22 04:29:08.080819: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-03-22 04:29:08.080833: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
I0000 00:00:1742628548.081396 1170733 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1742628548.081458 1170733 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        34,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,745 (311.50 KB)

...

 Non-trainable params: 960 (3.75 KB)

Epoch 1/200


2025-03-22 04:29:09.145471: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


380/380 ━━━━━━━━━━━━━━━━━━━━ 15s 33ms/step - loss: 37.0401 - mae: 5.8463 - val_loss: 18.4066 - val_mae: 4.1687 - learning_rate: 5.0000e-04
Epoch 2/200
380/380 ━━━━━━━━━━━━━━━━━━━━ 13s 34ms/step - loss: 13.9737 - mae: 3.3218 - val_loss: 2.0205 - val_mae: 1.0896 - learning_rate: 5.0000e-04
Epoch 3/200
380/380 ━━━━━━━━━━━━━━━━━━━━ 12s 32ms/step - loss: 3.5276 - mae: 1.3944 - val_loss: 1.0467 - val_mae: 0.6182 - learning_rate: 5.0000e-04
Epoch 4/200
380/380 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - loss: 2.7287 - mae: 1.2016 - val_loss: 0.9081 - val_mae: 0.5408 - learning_rate: 5.0000e-04
Epoch 5/200
380/380 ━━━━━━━━━━━━━━━━━━━━ 12s 32ms/step - loss: 2.3472 - mae: 1.1019 - val_loss: 0.8653 - val_mae: 0.5245 - learning_rate: 5.0000e-04
Epoch 6/200
380/380 ━━━━━━━━━━━━━━━━━━━━ 13s 34ms/step - loss: 2.1268 - mae: 1.0416 - val_loss: 0.8422 - val_mae: 0.5061 - learning_rate: 5.0000e-04
Epoch 7/200
380/380 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - loss: 1.9103 - mae: 0.9722 - val_loss: 0.8237 - val_mae: 0

KeyboardInterrupt: 